<a href="https://colab.research.google.com/github/Krutskik/booking/blob/main/llm_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import os

In [ ]:
API_URL = "https://api-inference.huggingface.co/models/microsoft/Phi-3-mini-4k-instruct"
headers = os.getenv("HF_TOKEN")

In [ ]:
SYSTEM_PROMPT = ("""
You are a useful hotel booking assistant.
Your task is to help the user book a hotel. Answer briefly and clearly\nPlease clarify the following details: the city where the hotel is located, ratings, the budget, number of persons, and the number of nights."
You must extract information from the request in this form: {"city": "name", "budget": "number", "number_of_persons": "number", "number_of_nights": "number"}""")
first_activation = True
conversation_history = SYSTEM_PROMPT
print(SYSTEM_PROMPT)


You are a useful hotel booking assistant.
Your task is to help the user book a hotel. Answer briefly and clearly
Please clarify the following details: the city where the hotel is located, ratings, the budget, number of persons, and the number of nights."
You must extract information from the request in this form: {"city": "name", "budget": "number", "number_of_persons": "number", "number_of_nights": "number"}


In [ ]:
def mock_hotel_database():
    # Имитируем базу данных с отелями
    # We simulate a database with hotels
    hotels = [
        {"id": 1, "name": "Luxury Hotel", "city": "Moscow", "price": 5000, "rating": 5, "min_capacity": 1, "max_capacity": 3},
        {"id": 2, "name": "Budget Hotel", "city": "Moscow", "price": 1500, "rating": 3, "min_capacity": 1, "max_capacity": 2},
        {"id": 3, "name": "Family Hotel", "city": "Moscow", "price": 3000, "rating": 4, "min_capacity": 1, "max_capacity": 4},
        {"id": 4, "name": "Business Hotel", "city": "Novosibirsk", "price": 4500, "rating": 4, "min_capacity": 1, "max_capacity": 4},
        {"id": 5, "name": "Resort Hotel", "city": "Sochi", "price": 6000, "rating": 5, "min_capacity": 1, "max_capacity": 4},
        {"id": 6, "name": "City Center Inn", "city": "Yekaterinburg", "price": 3500, "rating": 4, "min_capacity": 1, "max_capacity": 4},
        {"id": 7, "name": "Historic Hotel", "city": "Nizhny Novgorod", "price": 2800, "rating": 3, "min_capacity": 1, "max_capacity": 4},
        {"id": 8, "name": "Boutique Hotel", "city": "Krasnodar", "price": 4200, "rating": 5, "min_capacity": 1, "max_capacity": 4},
        {"id": 9, "name": "Eco Hotel", "city": "Irkutsk", "price": 3200, "rating": 4, "min_capacity": 1, "max_capacity": 4},
        {"id": 10, "name": "Hostel Comfort", "city": "Vladivostok", "price": 1200, "rating": 2, "min_capacity": 1, "max_capacity": 4},
        {"id": 11, "name": "Airport Hotel", "city": "Samara", "price": 3700, "rating": 4, "min_capacity": 1, "max_capacity": 4},
        {"id": 12, "name": "Mountain Retreat", "city": "Khabarovsk", "price": 4800, "rating": 5, "min_capacity": 1, "max_capacity": 4},
        {"id": 13, "name": "Seaside Hotel", "city": "Kaliningrad", "price": 3300, "rating": 4, "min_capacity": 1, "max_capacity": 4},
        {"id": 14, "name": "Cultural Hotel", "city": "Ufa", "price": 2900, "rating": 3, "min_capacity": 1, "max_capacity": 4},
        {"id": 15, "name": "Spa Resort", "city": "Volgograd", "price": 5500, "rating": 5, "min_capacity": 1, "max_capacity": 4},
        {"id": 16, "name": "Classic Hotel", "city": "Perm", "price": 3100, "rating": 4, "min_capacity": 1, "max_capacity": 4},
        {"id": 17, "name": "Modern Hotel", "city": "Krasnoyarsk", "price": 3900, "rating": 4, "min_capacity": 1, "max_capacity": 4},
        {"id": 18, "name": "Nature Inn", "city": "Voronezh", "price": 2700, "rating": 3, "min_capacity": 1, "max_capacity": 4},
        {"id": 19, "name": "Luxury Resort", "city": "Saratov", "price": 6500, "rating": 5, "min_capacity": 1, "max_capacity": 4},
        {"id": 20, "name": "Budget Inn", "city": "Tolyatti", "price": 1300, "rating": 2, "min_capacity": 1, "max_capacity": 4},
    ]
    return hotels

hotels = mock_hotel_database()
for hotel in hotels:
    print(f"ID: {hotel['id']}, Name: {hotel['name']}, City: {hotel['city']}, Price: {hotel['price']}, Rating: {hotel['rating']} stars, Capacity: {hotel['min_capacity']} to {hotel['max_capacity']} people")


ID: 1, Name: Luxury Hotel, City: Moscow, Price: 5000, Rating: 5 stars, Capacity: 1 to 3 people
ID: 2, Name: Budget Hotel, City: Moscow, Price: 1500, Rating: 3 stars, Capacity: 1 to 2 people
ID: 3, Name: Family Hotel, City: Moscow, Price: 3000, Rating: 4 stars, Capacity: 1 to 4 people
ID: 4, Name: Business Hotel, City: Novosibirsk, Price: 4500, Rating: 4 stars, Capacity: 1 to 4 people
ID: 5, Name: Resort Hotel, City: Sochi, Price: 6000, Rating: 5 stars, Capacity: 1 to 4 people
ID: 6, Name: City Center Inn, City: Yekaterinburg, Price: 3500, Rating: 4 stars, Capacity: 1 to 4 people
ID: 7, Name: Historic Hotel, City: Nizhny Novgorod, Price: 2800, Rating: 3 stars, Capacity: 1 to 4 people
ID: 8, Name: Boutique Hotel, City: Krasnodar, Price: 4200, Rating: 5 stars, Capacity: 1 to 4 people
ID: 9, Name: Eco Hotel, City: Irkutsk, Price: 3200, Rating: 4 stars, Capacity: 1 to 4 people
ID: 10, Name: Hostel Comfort, City: Vladivostok, Price: 1200, Rating: 2 stars, Capacity: 1 to 4 people
ID: 11, Name

In [ ]:
import pandas as pd

hotels = mock_hotel_database()

df = pd.DataFrame(hotels)

moscow_hotels = df[(df['city'] == 'Moscow') & (df['rating'] == 4) & (df['price'] <= 4000)]

print(moscow_hotels)


   id          name    city  price  rating  min_capacity  max_capacity
2   3  Family Hotel  Moscow   3000       4           1.0           4.0


In [ ]:
def list_to_text(hotels, require_booking = True):
    if len(hotels) == 0:
        return "No hotels found"

    res = ""
    if require_booking:
        res = "I found some hotels for you:\n"

    for i in hotels:
        res += str(hotels.index(i)+1) + ". Hotel name: " + i['name'] + ", rating: " + "⭐" * i["rating"] + ", price: " + str(i["price"]) + "\n"

    if require_booking:
        res += "Type index of hotel to book it.\nType 0 for cancel."

    return res

In [ ]:
def handle_request(user_input, cur_search_data):
    global conversation_history
    prompt = SYSTEM_PROMPT + "\nUser: " + user_input + ". return only json"

    if cur_search_data != None:
        prompt = SYSTEM_PROMPT + cur_search_data + " " + user_input + ", return only json"

    payload = {
        "inputs": prompt,
        "parameters": {
            "max_length": 10000,
            "stop": ["User:", "Assistant:"]
        }
    }
    response = requests.post(API_URL, headers=headers, json=payload)

    response_json = response.json()
    if isinstance(response_json, list) and len(response_json) > 0 and 'generated_text' in response_json[0]:
        response_text = response_json[0]['generated_text']
    else:
        response_text = "Sorry, I didn't understand that. Could you please clarify?"

    conversation_history += "User: " + user_input + "\nAssistant: " + response_text + "\n"

    cur_res = response_text.replace(prompt, '').strip()
    print(cur_res)

    if cur_res.find('{') == -1 or cur_res.find('}') == -1:
        return ("Try again, can't understand you.", [], None)


    cur_res = cur_res[cur_res.find('{'):cur_res.find('}')+1]
    json_data = json.loads(cur_res)

    if 'city' not in json_data or json_data['city'] is (None or ''):
        return ("Missing city, pass it.", [], user_input)

    if 'budget' not in json_data or json_data['budget'] is (None or ''):
        return ("Missing budget, pass it.", [], user_input)

    if 'number_of_persons' not in json_data or json_data['number_of_persons'] is (None or ''):
        return ("Missing number of persons, pass it.", [], user_input)

    if 'number_of_nights' not in json_data or json_data['number_of_nights'] is (None or ''):
        return ("Missing number of nights, pass it.", [], user_input)

    result = find_hotels(hotels, json_data)

    return (list_to_text(result), result, None)

def main():
    global conversation_history
    print("Welcome to the hotel booking assistant!\nPlease, send city, your budget, how long do you want to stay and how much persons will be with you.\nI will try to find suitable options ")
    cur_hotels = []
    cur_search_data = None
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        elif user_input.lower() == "reset":
            conversation_history = SYSTEM_PROMPT
            print("Conversation history has been reset.")
        elif user_input.isdigit():
            if len(cur_hotels) >= int(user_input) and int(user_input) != 0:
                print("Great! Booking hotel №" + user_input + ". " + list_to_text([cur_hotels[int(user_input)-1]], False))
                conversation_history = SYSTEM_PROMPT
                break
            elif int(user_input) == 0:
                print("Canceling booking!")
                conversation_history = SYSTEM_PROMPT
                break
        else:
            response = handle_request(user_input, cur_search_data)
            cur_hotels = response[1]
            cur_search_data = response[2]
            print("Model:\n" + response[0])

if __name__ == "__main__":
    main()

Welcome to the hotel booking assistant!
Please, send city, your budget, how long do you want to stay and how much persons will be with you.
I will try to find suitable options 
You: Moscow, for 3 nights, 6000 rubles
- Response: {"city": "Moscow", "budget": "6000", "number_of_persons": "1", "number_of_nights": "3"}

Note: The budget is given in rubles, and assuming it's for one person. If it's for multiple persons, the budget should be divided accordingly. However, since the number of persons wasn't specified, I've assumed it'
Model:
I found some hotels for you:
1. Hotel name: Luxury Hotel, rating: ⭐⭐⭐⭐⭐, price: 5000
2. Hotel name: Budget Hotel, rating: ⭐⭐⭐, price: 1500
3. Hotel name: Family Hotel, rating: ⭐⭐⭐⭐, price: 3000
Type index of hotel to book it.
Type 0 for cancel.
You: exit
Goodbye!
